<img src="../Images/logo.png" alt="RetroChem Logo" width="400">

The following Jupyter notebook briefly presents RetroChem, a pip-installable Python package designed for retrosynthetic analysis. This package was developed to assist chemists in predicting possible synthetic pathways for target molecules, using reaction templates and molecular fingerprints. Retrosynthesis is a central concept in organic chemistry, enabling the design of efficient synthetic routes by working backward from the desired product.

This package was created as a collaborative project for the EPFL course Practical Programming in Chemistry.

Before diving into the code and functionalities of the package, let’s briefly explore the motivations and core concepts that shaped its development.